# 9章 潜在顧客を把握するための画像認識10本ノック

In [1]:
# 下準備として，データのあるディレクトリに移動しておく
import os

DATA_ROOT_DIR = "./sample/9章/"
os.chdir(DATA_ROOT_DIR)

## ノック81 画像データを読み込んでみよう
まずはデータを読み込む．

In [2]:
import cv2
from IPython.display import Image, display
img = cv2.imread("img/img01.jpg")
height, width = img.shape[:2]
print("画像幅: " + str(width))
print("画像高さ: " + str(height))

# display(Image("img/img01.jpg"))

画像幅: 1920
画像高さ: 1440


## ノック８２：映像データを読み込んでみよう
やはりデータを読み込む．

In [3]:
import cv2
from IPython.display import Video, display, YouTubeVideo, HTML

# 情報取得
cap = cv2.VideoCapture("mov/mov01.avi")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print("画像幅: " + str(width))
print("画像高さ: " + str(height))
print("総フレーム数: " + str(count))
print("FPS: " + str(fps))

# 出力
# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret:
#         cv2.imshow("frame", frame)
#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break
cap.release()
# cv2.destroyAllWindows()

# display(Video("mov/mov01.avi", embed=True))

画像幅: 1920.0
画像高さ: 1440.0
総フレーム数: 401.0
FPS: 30.0


## ノック83 映像を画像に分割し、保存してみよう
動画から画像を切り抜く



In [4]:
import cv2
import os
cap = cv2.VideoCapture("mov/mov01.avi")
num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # cv2.imshow("frame", frame)
        filepath = "snapshot/snapshot_" + str(num) + ".jpg"
        if not os.path.exists(filepath):
            cv2.imwrite(filepath,frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    else:
        break
    num = num + 1
cap.release()
# cv2.destroyAllWindows()

## ノック84 画像内のどこに人がいるのかを検出してみよう
HOG特徴量を利用して人の検出を行う．

In [5]:
import cv2
from IPython.display import Image, display

# 準備
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'finalThreshold':5}

# 検出
img = cv2.imread("img/img01.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
human, r = hog.detectMultiScale(gray, **hogParams)
if (len(human)>0):
    for (x, y, w, h) in human:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255,255,255), 3)
# cv2.imshow("img",img)
_ = cv2.imwrite("temp.jpg",img)
# display(Image("temp.jpg"))

## ノック85 画像内の人の顔を検出してみよう
Haar-like特徴量を利用して顔検出を行う．

In [6]:
import cv2
from IPython.display import Image, display

# 準備
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

# 検出
img = cv2.imread("img/img02.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face_list = cascade.detectMultiScale(gray, minSize=(50, 50))

for (x, y, w, h) in face_list:
    color = (0, 0, 225)
    pen_w = 3
    cv2.rectangle(img, (x, y), (x+w, y+h), color, thickness = pen_w)

# cv2.imshow("img",img)
_ = cv2.imwrite("temp.jpg",img)
# cv2.waitKey(0)
# display(Image("temp.jpg"))

## ノック86 画像内の人がどこに顔を向けているのかを検出してみよう
dlibで顔のランドマークを検出し，顔の向きを推定する．

In [7]:
import cv2
import dlib
import math

# 準備 #
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# 検出 #
img = cv2.imread("img/img02.jpg")
dets = detector(img, 1)

for k, d in enumerate(dets):
    shape = predictor(img, d)

    # 顔領域の表示
    color_f = (0, 0, 225)
    color_l_out = (255, 0, 0)
    color_l_in = (0, 255, 0)
    line_w = 3
    circle_r = 3
    fontType = cv2.FONT_HERSHEY_SIMPLEX
    fontSize = 1
    cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), color_f, line_w)
    cv2.putText(img, str(k), (d.left(), d.top()), fontType, fontSize, color_f, line_w)

    # 重心を導出する箱を用意
    num_of_points_out = 17
    num_of_points_in = shape.num_parts - num_of_points_out
    gx_out = 0
    gy_out = 0
    gx_in = 0
    gy_in = 0
    for shape_point_count in range(shape.num_parts):
        shape_point = shape.part(shape_point_count)
        #print("顔器官No.{} 座標位置: ({},{})".format(shape_point_count, shape_point.x, shape_point.y))
        #器官ごとに描画
        if shape_point_count<num_of_points_out:
            cv2.circle(img,(shape_point.x, shape_point.y),circle_r,color_l_out, line_w)
            gx_out = gx_out + shape_point.x/num_of_points_out
            gy_out = gy_out + shape_point.y/num_of_points_out
        else:
            cv2.circle(img,(shape_point.x, shape_point.y),circle_r,color_l_in, line_w)
            gx_in = gx_in + shape_point.x/num_of_points_in
            gy_in = gy_in + shape_point.y/num_of_points_in

    # 重心位置を描画
    cv2.circle(img,(int(gx_out), int(gy_out)),circle_r,(0,0,255), line_w)
    cv2.circle(img,(int(gx_in), int(gy_in)),circle_r,(0,0,0), line_w)

    # 顔の方位を計算
    theta = math.asin(2*(gx_in-gx_out)/(d.right()-d.left()))
    radian = theta*180/math.pi
    print("顔方位:{} (角度:{}度)".format(theta,radian))

    # 顔方位を表示
    if radian<0:
        textPrefix = "   left "
    else:
        textPrefix = "   right "
    textShow = textPrefix + str(round(abs(radian),1)) + " deg."
    cv2.putText(img, textShow, (d.left(), d.top()), fontType, fontSize, color_f, line_w)


# cv2.imshow("img",img)
_ = cv2.imwrite("temp.jpg",img)
# display(Image("temp.jpg"))

顔方位:0.06456096931747406 (角度:3.6990710631648662度)


## ノック87 検出した情報を統合し、タイムラプスを作ってみよう
検出結果のタイムラプスを作成する．

In [8]:
from IPython.display import Video, display, YouTubeVideo, HTML
import cv2
import os

movie_name = "timelapse.avi"
if not os.path.exists(movie_name):
    print("タイムラプス生成開始")

    # 映像取得
    cap = cv2.VideoCapture("mov/mov01.avi")
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # hog
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'finalThreshold':5}

    # タイムラプス作成
    fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
    video = cv2.VideoWriter(movie_name,fourcc, 30, (width,height))

    num = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            if (num%10==0):
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                human, r = hog.detectMultiScale(gray, **hogParams)
                if (len(human)>0):
                    for (x, y, w, h) in human:
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,255), 3)

                video.write(frame)
        else:
            break
        num = num + 1
    video.release()
    cap.release()
    # cv2.destroyAllWindows()
    print("タイムラプス生成終了")
# display(Video("timelapse.avi", embed=True))